### Imports and data loading

In [1]:
import pandas as pd
import backtrader as bt
import datetime
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import warnings
from pandas.errors import SettingWithCopyWarning

%matplotlib inline

warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

# Set the path to the root directory
path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(path)

# Load Model 3 prediction file
df = pd.read_csv(path + '/data/predictions/12.csv')

# Quote_date between 2018-01-01 and 2018-01-31
df = df[(df['Quote_date'] >= '2018-01-01') & (df['Quote_date'] <= '2018-01-31')]

## Data processing

In [2]:
df['Expiry_date'] = pd.to_datetime(df['Quote_date']) + pd.to_timedelta(df['TTM'], unit='D')
df = df.dropna()

#### Adding ID column and new row for TTM = 0

In [3]:
# Sort the dataframe by Quote_date and Expiry_date
df = df.sort_values(['Quote_date', 'Expiry_date'])

# Create a dictionary to store the unique IDs for each group
group_id_dict = {}

# Enumerate over the unique groups to create a mapping from group to ID
for idx, group in enumerate(df.groupby(['Expiry_date', 'Strike']).groups.keys(), start=1):
    group_id_dict[group] = idx

# Use the apply method to assign IDs to each row based on their group membership
df['Option_ID'] = df.apply(lambda row: group_id_dict[(row['Expiry_date'], row['Strike'])], axis=1)

In [4]:
# Adding intrinsic value as Price on expirty date
groups = df.groupby(['Option_ID'])
for _, group in groups:
    # Create a new row for the date of expiry
    if group.iloc[-1]["TTM"] == 1 and pd.to_datetime(group.iloc[-1]["Expiry_date"] )< pd.to_datetime(df["Quote_date"]).max():
        new_row = group.iloc[-1].copy()
        new_row['Price'] = max(0, new_row['Underlying_last'] - new_row['Strike'])
        new_row['TTM'] = 0
        new_row['Quote_date'] = new_row['Expiry_date']
        df = pd.concat([df, new_row.to_frame().transpose()], ignore_index=True)

df_read = df.copy()


/var/folders/wk/x86_p6511l95p594k6qnb98h0000gn/T/ipykernel_11439/1592736345.py:3: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for _, group in groups:


### Parameters

In [5]:
buy_threshold = 5
sell_threshold = 9
starting_balance = 1000000

## Backtrader

In [6]:
df = df_read.copy()

# Filter Option_IDs between 1000 and 1100
df = df[(df['Option_ID'] >= 2678) & (df['Option_ID'] <= 2678)]

# Assuming df is your given dataframe
df['Close'] = df['Price']
df['Open'] = df['Close']
df['High'] = df['Close']
df['Low'] = df['Close']

df['Date'] = pd.to_datetime(df['Quote_date'])
df.set_index('Date', inplace=True)

# Add volume and open interest columns
df['Prediction'] = df['Prediction']
df['Option_ID'] = df["Option_ID"]

# Select only the required columns
backtrader_df = df[['Open', 'High', 'Low', 'Close', 'Prediction', 'Option_ID']]

backtrader_df

# Create custom datafeed class
class PredictionDataFeed(bt.feeds.PandasData):
    lines = ('Prediction', 'Option_ID')
    params = (
        ('Prediction', -1),
        ('Option_ID', -1),
    )
    
# Add a stratergy where we buy if price - volume > 1, and colse if it reaches 0
class OptionsStrategy(bt.Strategy):
    def __init__(self):
        self.dataclose = self.datas[0].close
        self.dataopen = self.datas[0].open
        self.datahigh = self.datas[0].high
        self.datalow = self.datas[0].low
        self.Prediction = self.datas[0].Prediction
        self.openinterest = self.datas[0].openinterest
    

    def next(self):
        if self.Prediction[0] - self.dataclose[0]  > buy_threshold:
            self.buy()
        elif self.Prediction[0] - self.dataclose[0] < sell_threshold:
            self.close()

# Create a backtrader Cerebro instance
cerebro = bt.Cerebro()
cerebro.addstrategy(OptionsStrategy)
feed = PredictionDataFeed(dataname=backtrader_df)
cerebro.adddata(feed)
cerebro.broker.setcash(starting_balance)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="trade_analyzer")
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio', riskfreerate=0.00, timeframe=bt.TimeFrame.Days, annualize=True)


# Run 
results = cerebro.run()
result = results[0]

print('Number of trades: {}'.format(result.analyzers.trade_analyzer.get_analysis()['total']['total']))
print('Number of winning trades: {}'.format(result.analyzers.trade_analyzer.get_analysis()['won']['total']))
print('Number of losing trades: {}'.format(result.analyzers.trade_analyzer.get_analysis()['lost']['total']))

print('Sharpe Ratio:', result.analyzers.sharpe_ratio.get_analysis()['sharperatio'])

# Plot
plt.rcParams['figure.figsize'] = [8, 9]
plt.rcParams['figure.dpi'] = 100
cerebro.plot(iplot=False)
plt.show()

# Get final portfolio Value
portvalue = cerebro.broker.getvalue()
pnl = portvalue - starting_balance

# Print out the final result
print('Final Portfolio Value: ${}'.format(portvalue))
print('P/L: ${}'.format(pnl))


Number of trades: 0


KeyError: 

In [ ]:
for option_ID in df["Option_ID"].

In [ ]:
# Try a bunch of different thresholds keep track of the best one in terms of sharpe ratio
best_sharpe_ratio = -100000
best_buy_threshold = 0
best_sell_threshold = 0

for buy_threshold in range(1, 10):
    for sell_threshold in range(1, 10):
        # Create a backtrader Cerebro instance
        cerebro = bt.Cerebro()
        cerebro.addstrategy(OptionsStrategy)
        feed = PredictionDataFeed(dataname=backtrader_df)
        cerebro.adddata(feed)
        cerebro.broker.setcash(starting_balance)
        cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="trade_analyzer")
        cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio', riskfreerate=0.01, timeframe=bt.TimeFrame.Days, annualize=True)


        # Run 
        results = cerebro.run()
        result = results[0]

        sharpe_ratio = result.analyzers.sharpe_ratio.get_analysis()['sharperatio']

        if sharpe_ratio > best_sharpe_ratio:
            best_sharpe_ratio = sharpe_ratio
            best_buy_threshold = buy_threshold
            best_sell_threshold = sell_threshold

print('Best Sharpe Ratio:', best_sharpe_ratio)

KeyboardInterrupt: 